# Call Centre Department

## Step 1: Import necessary packages

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)

## Step 2: Read all relevant CSVs

In [ ]:
df = pd.read_csv('../../Data/Call Centre Dept/Call_Centre_Merged_Raw.csv')
aventa_df = pd.read_csv('../../Data/Call Centre Dept/New Aventa.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
aventa_df

In [ ]:
aventa_df.info()

## Data Cleaning

### Step 3: Handle Missing Values

In [ ]:
df.notnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset=['Number'])

df.isnull().sum()

In [ ]:
aventa_df.notnull().sum()

In [ ]:
aventa_df.isnull().sum()

### Step 4: Drop Unwanted Columns

In [ ]:
df.drop(['Number','Actions'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.head()

### Step 5: Standardization of Data Types

In [ ]:
#Change 'Date' Column to datetime
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

#Show only Date format for 'Date'
#df['Date'] = pd.to_datetime(df['Date']).dt.date

In [ ]:
#Change 'Wait Time' & 'Talk Time' Column to datetime.dt.time
df['Wait Time'] = pd.to_datetime(df['Wait Time'], format='%H:%M:%S').dt.time
df['Talk Time'] = pd.to_datetime(df['Talk Time'], format='%H:%M:%S').dt.time

In [ ]:
df.isnull().sum()

## EDA

### Step 6: Match Functional Group (Team) based on Agent's Country and Queue's Team

#### Define Agent's Respective Country

In [ ]:
# Create column to check Agent Country (PSCS/PSCM)

ending = {'PSCS'}
mapper = {True: 'PSCS', False: 'PSCM'}

df['Country'] = df['Agent'].str[-4:].isin(ending).map(mapper)
df.head()

In [ ]:
#Edit column to check Agent == NONE, then Country == NONE
#df['Country'][df.Agent == 'NONE'] == "NONE"
df.loc[(df.Agent == "NONE"), 'Country'] = "NONE"

In [ ]:
#Functional Group by Agent_Country
#If Agent Name ends with 'Country', and Queue = Reporting_Aventa['Description'], append Country.

#aventa_df = pd.read_csv('../Reporting_Aventa_edited.csv', usecols=['Description', 'Team', 'Country'])

#dfinal_agent = pd.merge(df, aventa_df, how='left', left_on=['Queue', 'Agent_Country'], right_on=['Description', 'Country'])

#change 'Team' to 'Agent_Team'
#dfinal_agent.rename(columns={'Team':'Agent_Team'}, inplace=True)

#dfinal_agent.head()



In [ ]:
#Check Queue with no Functional Group (Team)
#dfnull_agent = dfinal_agent[dfinal_agent.isna().any(axis=1)]
#dfnull_agent

In [ ]:
#Check for unique values of those with no Functional Group (Team)
#dfnull_agent['Queue'].unique()

In [ ]:
#is_GEH_Day = dfnull_agent['Queue']=='GEH Internal - Night'
#dfnull_agent_is_GEH_Day = dfnull_agent[is_GEH_Day]
#print(dfnull_agent_is_GEH_Day.head())

#### Define Queue's Respective Team

In [ ]:
#Change all Queue: 'Hotel House call Services-Day' to same as Description: 'Hotel House Call Services - Day'

df.loc[(df['Queue'] == 'Hotel House call Services-Day'), 'Queue'] = 'Hotel House Call Services - Day'

In [ ]:
dfinal = pd.merge(df, aventa_df, how='left', left_on=['Queue'], right_on=['Description'])

#change 'Team' to 'Queue_Team'
#dfinal.rename(columns={'Team':'Queue_Team'}, inplace=True)

dfinal.head()

In [ ]:
#check if there is any NaN
dfnull_team = dfinal[dfinal.isna().any(axis=1)]
dfnull_team

In [ ]:
dfnull_team['Queue'].unique()

### Step 7: Get Calls Offered

In [ ]:

dfinal['Calls_Offered'] = np.where((dfinal['Event'] == 'COMPLETEAGENT') | (dfinal['Event'] == 'COMPLETECALLER') | (dfinal['Event'] == 'ABANDON'), 1, 0)
dfinal.head()

In [ ]:
print(dfinal[dfinal['Calls_Offered']==0].head())

### Step 8: Get Calls Answered

In [ ]:
dfinal['Calls_Answered'] = np.where((dfinal['Event'] == 'COMPLETEAGENT') | (dfinal['Event'] == 'COMPLETECALLER'), 1, 0)

dfinal.head()

In [ ]:
print(dfinal[dfinal['Calls_Answered']==0].head())

### Step 9: Get Waiting Time <= 20 seconds and <= 30 seconds

Dataframe carried forward from Calls Answered

In [ ]:
from datetime import time

twenty_sec = time(0,0,20,0)
thirty_sec = time(0,0,30,0)

dfinal['Wait_Time_<=20s'] = np.where((dfinal['Event'] != 'ABANDON') & (dfinal['Wait Time'] <= twenty_sec),1,0)
dfinal['Wait_Time_<=30s'] = np.where((dfinal['Event'] != 'ABANDON') & (dfinal['Wait Time'] <= thirty_sec),1,0)

dfinal.head()

In [ ]:
print(dfinal[dfinal['Wait_Time_<=20s']==0].head())

In [ ]:
print(dfinal[dfinal['Wait_Time_<=30s']==0].head())

### Step 10: Get Day of Week

Dataframe carried forward from Waiting Time <= 20s and <=30s

In [ ]:
dfinal['Weekday'] = dfinal['Date'].dt.day_name()

dfinal.head()

### Step 11: Understand if a call is Inbound or Outbound

Dataframe carried forward from Day of Week

In [ ]:
dfinal['Inbound_Outbound'] = np.where((dfinal['Queue']) != "PSCS-Outbound", "Inbound", "Outbound")

dfinal.head()

In [ ]:
print(dfinal[dfinal['Inbound_Outbound']=="Outbound"].head())

## Step 12: Export to CSV for PowerBI

In [ ]:
dfinal.to_csv('../../Data/Call Centre Dept/Call_Centre_CLEANED_Final.csv', index = False)